In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os

In [2]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [4]:
import json
import pickle
import re
from tqdm import tqdm
from pathlib import Path

### 0.API

In [5]:
# id_standard_df.to_csv('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/id2class99_chenyi.csv', index=False)

In [6]:
id_standard_df = pd.read_csv('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/id2class99_chenyi.csv')
id_use_df = id_standard_df[id_standard_df['use']>=0]

In [7]:
with open('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/haowei34k_map_stand.json', 'r') as f:
    hw2stand = json.load(f)

In [8]:
shbg_map = pd.read_excel('./data/shoe_bag11k_map.xlsx')

In [9]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [10]:
def write_txt(input_list):
    label_path, text = input_list[0], input_list[1]
    with open(label_path, 'a+') as f:
        f.write(text)

In [11]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [12]:
l2_map_l1 = get_list2dict( id_use_df['label99'].values, id_use_df['label_l1'].values)

In [13]:
id2chi_l2 = get_list2dict(id_use_df['label2_chi'].values, id_use_df['label99'].values)

In [14]:
id2eng_l2 = get_list2dict(id_use_df['label99'].values, id_use_df['label2_eng'].values)

In [15]:
id2chi_l1 = get_list2dict(id_use_df['label_l1'].values, id_use_df['label1_chi'].values)

In [16]:
id2eng_l1 = get_list2dict(id_use_df['label_l1'].values, id_use_df['label1_eng'].values)

In [17]:
id2model_l1 = get_list2dict(model_label_map.values(), model_label_map.keys())

In [18]:
shbg_map_dict = get_list2dict(shbg_map[shbg_map.columns[0]], shbg_map[shbg_map.columns[1]])

### 1.Dataset Information.

In [19]:
import numpy as np

In [20]:
df = pd.read_csv('./data/shoe_bag11k_Jul.csv')

In [21]:
# p = '/home/chenyi/workspace/myPyModule99/data_format/haowei34k_voc_bboxes_relabel.csv'
# openfashion_pd.to_csv(p, index=False)

In [22]:
df['label_model'] = -1
df['label_model'] =  df[df.columns[3]].apply(lambda x: shbg_map_dict[str(x)])

In [23]:
df = df[df['label_model']>=0]

### 2.统计数据集

In [24]:
df.head()

,0,1,2,3,4,5,6,7,man,stem,path,label_model
0,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,3024,4032,包,685,719,2231,3806,lifengju,2018053014441155222,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
1,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,800,800,包,206,1,650,778,lifengju,2018031823531012935,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
2,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,800,800,包,197,103,724,431,lifengju,2018050611334121830,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
3,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,750,750,包,74,2,698,724,lifengju,2018050416361575803,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
4,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,800,800,包,76,198,755,731,lifengju,2018030623232873287,/home/chenyi/workspace/dataset/data/dada/shoe_...,1


In [25]:
df.value_counts('label_model')

label_model
0    8859
1    5022
2     115
3      58
4      42
5      23
dtype: int64

#### 二级标签

In [ ]:
openfashion_pd.head(2)

In [ ]:
openfashion_pd[openfashion_pd['label8']==5].value_counts('label43')

In [ ]:
counter = openfashion_pd.value_counts('label_num').sort_index()

In [ ]:
id_list = np.array(counter.axes)[0]
numbers = counter.values
name_list = [str(x) + ':' + str(id2eng_l2[hw2stand[str(x)][0]]) for x in id_list]

In [ ]:
plt.figure(figsize=(25, 7))
plt.xticks(list(range(len(name_list))), name_list, rotation=75, fontsize=14)
plt.bar(name_list, numbers, width=0.5)  # 绘制y刻度标签
for a, b in zip(name_list, numbers):
    plt.text(a, b, b, ha='center', va='bottom')
plt.show()

#### 一级标签

In [ ]:
openfashion_pd.head()

In [ ]:
counter = openfashion_pd.value_counts('label8').sort_index()

In [ ]:
id_list = np.array(counter.axes)[0]
numbers = counter.values
name_list = [str(x) + ':' + id2eng_l1[x] for x in id_list]

In [ ]:
plt.figure(figsize=(25, 7))
plt.xticks(list(range(len(name_list))), name_list, rotation=75, fontsize=14)
plt.bar(name_list, numbers, width=0.5)  # 绘制y刻度标签
for a, b in zip(name_list, numbers):
    plt.text(a, b, b, ha='center', va='bottom')
plt.show()

#### Haowei34k_conf

In [ ]:
!ls /nas/chenyi/dataset_apparel_nas/

In [ ]:
hw_data_p = '/nas/chenyi/dataset_apparel_nas/haowei34k_conf7.csv'

In [ ]:
hw_data_df = pd.read_csv(hw_data_p)

In [ ]:
hw_data_df.head()

In [ ]:
counter_hw = hw_data_df['label_2'].value_counts()

In [ ]:
id_list = np.array(counter_hw.axes)[0]
numbers = counter_hw.values
name_list = [str(x) + ':' + names[x][1] for x in id_list]

In [ ]:
plt.figure(figsize=(25, 7))
plt.xticks(list(range(len(name_list))), name_list, rotation=50, fontsize=14)
plt.bar(name_list, numbers, width=0.5)  # 绘制y刻度标签
for a, b in zip(name_list, numbers):
    plt.text(a, b, b, ha='center', va='bottom')
plt.show()

### target label

In [ ]:
df = openfashion_pd[openfashion_pd['label8']>=0].copy(deep=True)

In [ ]:
len(df),len(openfashion_pd)

In [ ]:
df['label_model'] = -1

In [ ]:
df['label_model'] = df['label8'].map(lambda x: id2model_l1[x])

#### target label path

In [ ]:
!ls /home/chenyi/workspace/dataset/hw_pedia_true/

In [ ]:
root = '/home/chenyi/workspace/dataset/hw_pedia_may/labels6/'

In [ ]:
df['label_tpath'] = df.apply(lambda row: Path('{0}{1}/{2}.txt'.format(root, row['mode'], row['fname'])), axis=1)

In [73]:
df.head()

,0,1,2,3,4,5,6,7,man,stem,path,label_model
0,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,3024,4032,包,685,719,2231,3806,lifengju,2018053014441155222,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
1,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,800,800,包,206,1,650,778,lifengju,2018031823531012935,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
2,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,800,800,包,197,103,724,431,lifengju,2018050611334121830,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
3,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,750,750,包,74,2,698,724,lifengju,2018050416361575803,/home/chenyi/workspace/dataset/data/dada/shoe_...,1
4,/nas/chenyi/datasets_nas/dada_labelme/labelme_...,800,800,包,76,198,755,731,lifengju,2018030623232873287,/home/chenyi/workspace/dataset/data/dada/shoe_...,1


In [33]:
!ls /nas/chenyi/

benchmark  code  datasets_cls  datasets_nas  scripts_data  share  tools


#### 生成txt

In [26]:
sys.path.append('../../')

In [27]:
from utils.bbox.conversation import bbox_coco2voc,bbox_voc2yolo,bbox_yolo2voc

In [28]:
df.columns[]

Index(['0', '1', '2', '3', '4', '5', '6', '7', 'man', 'stem', 'path',
       'label_model'],
      dtype='object')

In [29]:
bboxes_list = []
for ind,row in tqdm(df.iterrows()):
    bbox7 = row.loc[['label_model', '4', '5', '6', '7', '1', '2']].values.tolist()
    bbox5 = bbox7[:6]
    bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
    bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
    bboxes_list.append(bbox5_str)
    break

0it [00:00, ?it/s]


In [ ]:
%%time
input_list = df.apply(lambda x: 
                        [x['label_sp'], 
                         '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(
                             *x.loc[['label8', 'bx1','bx2', 'bx3', 'bx4',]])], 
                        axis=1)

In [ ]:
def write_txt(label_path, text):
    # label_path, text = input_list[0], input_list[1]
    label_path = Path(label_path)
    p_parent = label_path.parent
    if not p_parent.is_dir():
        p_parent.mkdir()
    if not p_parent.is_file():
        with open(label_path, 'a+') as f:
            f.write(text)

In [ ]:
for label_path, text in tqdm(zip(df['label_tpath'].values, bboxes_list)):
    write_txt(label_path, text)
    # break

In [ ]:
label_path,text

In [ ]:
!cat $label_path

#### 生成软连接的path

In [ ]:
!tree -d L 2 /home/chenyi/workspace/dataset/data/haowei34k/images/

In [ ]:
!tree -d L 2 /home/chenyi/workspace/yolov5/datasets/hw_pedia_may/

In [ ]:
# !mkdir -p /home/chenyi/workspace/yolov5/datasets/hw_pedia_may/labels/train

In [ ]:
drop_df = df.drop_duplicates('label_tpath')

In [ ]:
slink_df = pd.DataFrame()
slink_df['fname'] = drop_df['fname']
slink_df['label_spath'] = drop_df['label_tpath']

In [ ]:
root = Path('/home/chenyi/workspace/yolov5/datasets/hw_pedia_true/')
data_root = Path('/home/chenyi/workspace/dataset/data/haowei34k/')

In [ ]:
slink_df['label_tpath'] = slink_df['label_spath'].apply(lambda x: root /'labels' /x.parts[-2]/x.parts[-1])

In [ ]:
slink_df['image_tpath'] = slink_df['label_spath'].apply(lambda x: root /'images' /x.parts[-2]/x.parts[-1])
slink_df['image_tpath'] = slink_df['image_tpath'].apply(lambda x: x.with_suffix('.jpg'))

In [ ]:
slink_df['image_spath'] = slink_df['image_tpath'].apply(lambda x: troot /'images' /x.parts[-2]/x.parts[-1])

#### 直接读取slink.csv

In [ ]:
slink_df = pd.read_csv('./data/hw34k_level1_slink.csv')

In [ ]:
slink_df.head(2).values

In [ ]:
slink_df['image_tpath'] = slink_df['image_tpath'].apply(lambda x:x.replace('/datasets/hw_pedia_may/images/', '/datasets/hw_pedia_true/images/'))

In [ ]:
slink_df['label_tpath'] = slink_df['label_tpath'].apply(lambda x:x.replace('/datasets/hw_pedia_may/labels/', '/datasets/hw_pedia_true/labels/'))

In [ ]:
slink_df.head(2)

#### 生成sym link

In [ ]:
!tree  -d -L 2 /home/chenyi/workspace/yolov5/datasets/hw_pedia_true/

In [ ]:
slink_df.head(2)

In [ ]:
%%time
# _ = slink_df.apply(lambda row: Path.li(row['image_spath'], row['image_tpath']), axis=1)
_ = slink_df.apply(lambda row: Path.link_to(Path(row['image_spath']), Path(row['image_tpath'])), axis=1)

In [ ]:
%%time
_ = slink_df.apply(lambda row: Path.link_to(Path(row['label_spath']), Path(row['label_tpath'])), axis=1)

In [ ]:
len(glob.glob('/home/chenyi/workspace/yolov5/datasets/hw_pedia_true/images/*/*.jpg'))

In [ ]:
len(glob.glob('/home/chenyi/workspace/yolov5/datasets/hw_pedia_true/labels/*/*.txt'))

In [ ]:
p = str(slink_df['label_tpath'][0])
!cat $p

In [ ]:
Image.open(slink_df['image_tpath'][0])